<a href="https://colab.research.google.com/github/nerofinal012/Cisco-Dataset-Training/blob/main/Cisco_Dataset_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install --upgrade transformers

from transformers import pipeline, Conversation
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, pipeline
import pandas as pd
import torch

# Verify GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
    print(torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

# Load the dataset from Hugging Face
cisco_dataset = load_dataset("ndavidson/cisco_dcnm_finetuning_chatml", split="train")
print(cisco_dataset.column_names)  # Inspect the columns

GPU is available
NVIDIA L4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['Question', 'Text', 'Answer', 'prompt_and_answer']


In [2]:
# Create a DataFrame from the dataset
df_cisco = pd.DataFrame(cisco_dataset)

In [3]:
# Preprocessing: Add input and response columns
df_cisco['input'] = df_cisco['Question']
df_cisco['response'] = df_cisco['Answer']

# Display the first few rows to verify
print(df_cisco[['input', 'response']].head())

                                               input  \
0  what is the latest version of Nexus Dashboard ...   
1  newest Nexus Dashboard Fabric Controller version?   
2  what's current Nexus Dashboard Fabric Controll...   
3   latest Nexus Dashboard Fabric Controller update?   
4     Nexus Dashboard Fabric Controller version now?   

                                            response  
0  The latest version of Nexus Dashboard Fabric C...  
1  Based on the information provided in the sourc...  
2  Based on the information provided in the sourc...  
3  Based on the information provided in the sourc...  
4  The current version of the Nexus Dashboard Fab...  


In [4]:
# Choose a pre-trained model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Assign a padding token
tokenizer.pad_token = tokenizer.eos_token

# Define the tokenization function
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], truncation=True, padding='max_length', max_length=512)
    labels = tokenizer(examples['response'], truncation=True, padding='max_length', max_length=512)
    inputs["labels"] = labels["input_ids"]
    return inputs

# Convert the DataFrame back to a Dataset object
cisco_dataset = Dataset.from_pandas(df_cisco)

# Apply the tokenization function
tokenized_datasets = cisco_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2506 [00:00<?, ? examples/s]

In [5]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Further reduce batch size
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=8, # Increase gradient accumulation steps
)

In [6]:
# Subclassing Trainer to Override compute_loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Create the Trainer instance with the custom compute_loss method
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets
)

In [7]:
# Train the Model
trainer.train()

# Evaluate the Model
results = trainer.evaluate()
print(results)

Epoch,Training Loss,Validation Loss
0,No log,3.019367
1,3.303900,2.929858
2,3.303900,2.858711


{'eval_loss': 2.858710527420044, 'eval_runtime': 169.0571, 'eval_samples_per_second': 14.823, 'eval_steps_per_second': 1.857, 'epoch': 2.997605746209098}


In [8]:
# Deploy the Chatbot
chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)

# Test the chatbot
conversation = Conversation("What is the latest version of the Nexus Dashboard?")
response = chatbot(conversation)
print(response)

Conversation id: 2a77eeb9-79b4-4db4-82f4-1070fcdee26d
user: What is the latest version of the Nexus Dashboard?
assistant: 

